In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_train.shape

In [ ]:
y_train = y_train.reshape(-1,)
y_train[:5]

In [ ]:
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
def plot_sample(X, y, index):
  plt.figure(figsize=(15,2))
  plt.imshow(X[index])
  plt.xlabel(classes[y[index]])

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report

# Cell mới: Tải và sửa đổi mô hình VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Đóng băng các lớp cơ sở

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # 10 lớp cho CIFAR-10
])

# Cell mới: Tiền xử lý dữ liệu cho VGG16 (resize và chuẩn hóa)
def preprocess_for_vgg(X):
    X_resized = tf.image.resize(X, (224, 224))
    return preprocess_input(X_resized)

X_train_vgg = preprocess_for_vgg(X_train)
X_test_vgg = preprocess_for_vgg(X_test)

# Cell mới: Biên dịch và huấn luyện mô hình
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_vgg, y_train, epochs=5, validation_data=(X_test_vgg, y_test))

# Cell mới: Đánh giá và dự đoán
loss, accuracy = model.evaluate(X_test_vgg, y_test)
print(f"Độ chính xác trên tập test: {accuracy:.4f}")

y_pred = model.predict(X_test_vgg)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes, target_names=classes))

# Cell mới: Hiển thị dự đoán mẫu (tùy chọn)
plot_sample(X_test, y_test, 0)  # Hiển thị ảnh gốc
print(f"Dự đoán: {classes[y_pred_classes[0]]}")